In [1]:
#Current Problem

#child problems are depenting on each other, so need to solve them in order and asign tasks to the correct child problem and vice versa

#tasksolver sometimes doesnt only need result of direct childs, but also of indirect childs (childs of childs)

#results are added in the wrong order in dummySolver

#an return all usefull information in this text for this task( and the context of the taskpath)

# https://www.reddit.com/r/AutoGPT/comments/12k4al2/found_the_winning_sauce/

In [2]:
#we gone need many types of BrainFunctions

#Tasks
    #tasks have subtasks
    #if a task couldn't be solved, it can be retried later
    #tasks should consist of descriptions results and parameters

#taskpath 
    #Path from Objectiv Task to current Task. Creates a great context for the current task.

#Atomic Tasks are tasks with no subtasks

#Central control
#    decides what to do next.
#       Split Tast
#       Solve Task
#       Retry Task
#       Improve Task Solution

#TaskSplitter Task->List[Task] (simpler tasks)
#TaskSolver Task->Result + Optional List[Task] (new tasks encountered during solving, debugging, etc.)
#Task Duplicate Detector List[Task]->List[Task]
#Task Merger List[Task]->Task
#Task Selector 
#Function tath gathers all the results that can be helpfull to solve a task
#asign solved task to other tasks that could benefit from the results
#ask for most complex task and split it into subtasks


#JSON Stuffer Takes result of Brainfunction and stuffs it into a JSON string
#retries with "doesnt look like correct JSNO jet" until its successfull parsed into object

In [32]:
import json
import re
from collections import deque
from typing import Dict, Iterable, List, Set, Optional, Any

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain

from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

import os
from langchain.chat_models import PromptLayerChatOpenAI
from langchain.schema import HumanMessage

class Task:
    def __init__(self, TaskDescription: str = "dscr", Result:str = "not solved", Solved:bool = False):
        self.Description:str = TaskDescription
        self.Result:str = Result
        self.Solved:bool = Solved
        self.ChildTasks:List[Task] = []
        self.ParentTasks:List[Task] = []
        
    def __str__(self)->str:
        if(self.Solved):
            return "Solved["+self.Description+"]>>"+self.Result
        else:
            return "Unsolved["+self.Description+"]>>"+self.Result
        
    def get_recusively_all_subtasks(self, visited: Set['Task'] = None) -> List['Task']:
        if visited is None:
            visited = set()
        if self in visited:
            return []
        visited.add(self)
        all_tasks = [self]  # Add the current task to the list
        for subtask in self.ChildTasks:
            all_tasks +=subtask.get_recusively_all_subtasks(visited)  # Recursively get all subtasks of the subtask
        return all_tasks

    def display(self, indent_level: int = 0) -> None:
        indent = '  ' * indent_level  # Modify the number of spaces in the quotes to adjust indentation
        print(indent + str(self))
        for subtask in self.ChildTasks:
            subtask.display(indent_level + 1)
    
    def is_atomic(self)->bool:
        return len(self.ChildTasks) == 0
    
    def ready_to_solve(self)->bool:
            return all(map(lambda x: x.Solved, self.ChildTasks)) #all([]) is True
        
    def add_child(self, child: 'Task') -> None:
        if child in self.ChildTasks:
            return
        self.ChildTasks.append(child)
        child.ParentTasks.append(self)

    def add_childs(self, childs: List['Task']) -> None:
        for child in childs:
            self.add_child(child)

class Taskpool(List[Task]):    
    def __str__(self)->str:
        return ",".join(map(str, self.tasks))
    
    def CreateContextSummery(self)->str:
        return f"/n".join(map(lambda x: x.TaskDescription, self.tasks))
    
    def get_task_ready_to_solve(self)->Task:
        for task in self:
            if(task.Solved):
                continue
            if(task.ready_to_solve()):
                return task
        raise Exception("No task ready to solve")
    
    def get_nummerated_task_List(self) -> str:
        formatted_tasks = []
        for index, task in enumerate(self):
            formatted_tasks.append(f"{index}:{task.Description}")
        return "\n".join(formatted_tasks)
    

class Agent():
    def __init__(self, Objectiv: str):
        self.Objectiv:Task = Task(Objectiv)
        self.Taskpool = Taskpool()
        self.Taskpool.append(self.Objectiv)
        # self.chat = ChatOpenAI(temperature=0)
        self.chat = PromptLayerChatOpenAI(temperature=0)

    def extract_json(self,text):
        # Find a JSON object or array pattern in the text
        pattern = r'({.*?}|\[.*?\])'
        match = re.search(pattern, text, re.DOTALL)

        # Check if a match is found
        if match:
            json_str = match.group()
            try:
                # Parse the JSON string into a Python object
                json_obj = json.loads(json_str)
                return json_obj
            except json.JSONDecodeError:
                print("Invalid JSON detected.")
                return None
        else:
            print("No JSON found in the text.")
            return None

    def should_be_split(self, task: Task) -> bool:
        #test if task is atomic if no throw exception
        if len(task.ChildTasks) != 0:
            raise ValueError("Task is not atomic")
        
        print("Now in should_be_slpit:" + task.Description)

        
       

        messages = []
        messages.append(SystemMessage(content=
                                      "You are an AI that determines if a task could be split into multiple simpler tasks. Return 'True' if it could be split and 'False' otherwise. Dont be chaty."))
        
        messages.append(HumanMessage(content=
                                     f"Return 'True' if it could be split and 'False' otherwise. Task description: {task.Description}"))
        
        max_retries = 3
        for _ in range(max_retries):
            response = self.chat(messages).content
            clean_response = response.strip().lower()

            if 'true' in clean_response:
                return True
            elif 'false' in clean_response:
                return False
            else:
                messages.append(HumanMessage(content="Please provide a valid response with either 'True' or 'False'."))
        raise ValueError("AI response does not contain a valid boolean string after multiple retries: " + response)

    def SplitTask(self, task: Task, verbose: bool = True) -> List[Task]:
        def collect_parent_descriptions(task: Task, visited: Set['Task'] = None) -> List[str]:
            if visited is None:
                visited = set()
            if task in visited:
                return []
            visited.add(task)
            descriptions = []
            for parent_task in task.ParentTasks:
                descriptions.append(parent_task.Description)
                descriptions.extend(collect_parent_descriptions(parent_task, visited))
            return descriptions

        print("Now in SplitTask:" + task.Description)

        messages = []

        messages.append(SystemMessage(content="""You are a task-splitting AI. Your goal is to analyze the main task provided and break it down into smaller, independent, and more manageable subtasks that contribute to its successful completion. Keep your response concise and focused. Ensure the subtasks are independent, can be completed in any order, and directly help solve the main task. Create a hierarchical view, starting with the tasks that need to be completed first. Do not expand the scope of the task with the subtasks and take the task literally."""))

        # Check if the task has any parent tasks
        parent_descriptions = collect_parent_descriptions(task)
        if parent_descriptions:
            # Include the descriptions of the parent tasks in the task description
            task_description = f"Maintask: {task.Description}. The context of the parent tasks is: {', '.join(parent_descriptions)}"
            task_description += "\n Use this context to split the main task into smaller, independent, and more manageable subtasks."
            messages.append(HumanMessage(content=task_description))
        else:
            messages.append(HumanMessage(content="""As a task-splitting AI, analyze the main task provided and break it down into smaller, independent, and more manageable subtasks that contribute to its successful completion. Ensure the subtasks are independent, can be completed in any order, and directly help solve the main task. Create a hierarchical view, starting with the tasks that need to be completed first. Do not expand the scope of the task with the subtasks and take the task literally. Main task to split: """ + task.Description))

        # Call OpenAI
        generation = self.chat(messages)

        print("Splitted Tasks before JSON packing: " + generation.content)

        messages.append(generation)

        messages.append(HumanMessage(content="""Don't expand the scope of the task with the subtasks, take the task literally. Keep the scope as minimal as possible."""))

        # Call OpenAI
        response = self.chat(messages)

        messages.append(response)

        messages.append(HumanMessage(content="""Now transform this List of Tasks to a JSON Array of strings like ["hello", "world"]."""))

        # Call OpenAI
        JSONSTR = self.chat(messages).content
        print("Splitted Tasks as JSON " + JSONSTR)

        JSON = self.extract_json(JSONSTR)

        Tasks = []

        for subtaskstr in JSON:
            Tasks.append(Task(TaskDescription=subtaskstr))

        # Link the tasks as child tasks in a sequential manner, each task is a child task of the task before it
        Tasks.reverse()  # Last Task depends on Tasks before it so he is the first with a child
        task.add_child(Tasks[0])
        for i in range(len(Tasks) - 1):
            Tasks[i].add_child(Tasks[i + 1])

        return Tasks
    
    def DummySolveAtomicTask(self, task: Task, max_retries: int = 5, verbose: bool = True) -> Task:
        def collect_parent_descriptions(task: Task, visited: Set['Task'] = None) -> List[str]:
            if visited is None:
                visited = set()
            if task in visited:
                return []
            visited.add(task)
            descriptions = []
            for parent_task in task.ParentTasks:
                descriptions.append(parent_task.Description)
                descriptions.extend(collect_parent_descriptions(parent_task, visited))
            return descriptions

        # Loop until a satisfying solution is found or the maximum number of retries is reached
        for i in range(max_retries):
            finalMessages = []
            finalMessages.append(SystemMessage(content="You are a dummy task solver AI to test systems. Your goal is to simulate the solution to a task and provide believable information and results without being chatty. Focus on the results, and remember that the information's accuracy is not important as long as it appears to be true."))

            # Check if the task has any parent tasks
            parent_descriptions = collect_parent_descriptions(task)
            if parent_descriptions:
                # Include the descriptions of the parent tasks in the task description
                task_description = f"Current Task: {task.Description}. \n This Current Task will be used to solve the parent tasks. Keep that in mind while solving it.\n\n Parent Tasks:  {', '.join(parent_descriptions)}"
                finalMessages.append(HumanMessage(content=task_description))

            # Check if the task has any child tasks that have been solved
            childs = task.get_recusively_all_subtasks()
            if len(childs)>0:
                # Include the results of the child tasks in the task description
                task_description = f"Current Task: {task.Description}. The Current Task has Subtasks that are solved. The results and descriptions of the child tasks are: " + ', '.join(map(lambda t:(t.Description + ": " + t.Result),  childs))
                task_description += "\n Use these results to solve the current task."
                finalMessages.append(HumanMessage(content=task_description))

            finalMessages.append(HumanMessage(content=f"Current Task: {task.Description}"))

            # Get a chat completion from the formatted messages
            solutionMessage = self.chat(finalMessages)
            finalMessages.append(solutionMessage)
            # Ask the AI if the solution is satisfying
            finalMessages.append(HumanMessage(content=f"Is this solution satisfying? Was your work successful, answer with 'True' or 'False'"))

            # Get the AI's response
            ai_response = self.chat(finalMessages).content.strip().lower()

            if 'true' in ai_response:
                task.Result = solutionMessage.content
                task.Solved = True
                return task
            elif 'false' in ai_response:
                print("AI response was not satisfying, retrying...")
            elif 'true' not in ai_response and 'false' not in ai_response:
                print("AI response does not contain a valid boolean string.")
            else:
                raise ValueError("AI response does not contain a valid boolean string: " + ai_response)

        raise ValueError("Failed to find a satisfying solution after multiple retries.")
    
    
    def run(self):
        
        #hardcoded passes of possible splitting
        for i in range(1):
            currentTasks = Taskpool()
            currentTasks.extend(self.Taskpool) #create copy of Taskpool or else loop will be endless
            for task in currentTasks:
                if task.is_atomic():
                    #if self.should_be_split(task):
                    splitDone=True
                    print("Splitting Task")
                    newTasks = self.SplitTask(task)
                    self.Taskpool.extend(newTasks)
                    self.Objectiv.display()
                    print("Splitting Done")
        
        print(self.Taskpool.get_nummerated_task_List())

        while self.Objectiv.Solved==False:
            currentTask=self.Taskpool.get_task_ready_to_solve()
            self.DummySolveAtomicTask(currentTask)
            self.Objectiv.display()

In [33]:
agent = Agent("Write a story about a fun adventure by the cast of Frends(Sitcome) that takes place in the Frends(Sitcome) universe.")
agent.run()
print(agent.Objectiv.Result)





Splitting Task
Now in SplitTask:Write a story about a fun adventure by the cast of Frends(Sitcome) that takes place in the Frends(Sitcome) universe.
Splitted Tasks before JSON packing: 1. Familiarize with the Friends universe and characters
2. Brainstorm ideas for the adventure story
3. Choose the main characters for the adventure
4. Develop the plot for the adventure story
5. Create a setting for the adventure story
6. Write the first draft of the adventure story
7. Edit and revise the adventure story
8. Proofread the adventure story for errors
9. Finalize the adventure story
10. Submit the adventure story for review.
Splitted Tasks as JSON ["Brainstorm ideas for the adventure story", "Choose the main characters for the adventure", "Develop the plot for the adventure story", "Create a setting for the adventure story", "Write the first draft of the adventure story", "Edit and revise the adventure story", "Proofread the adventure story for errors", "Finalize the adventure story"]
Unsolv